In [22]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print(train_images.ndim)
print(train_images.shape)
print(len(train_labels))
my_slice = train_images[:,14:, 14:]
plt.imshow(train_images[0])
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255


3
(60000, 28, 28)
60000
Error in callback <function flush_figures at 0x7fa5841498a0> (for post_execute):


KeyboardInterrupt: 

In [ ]:
from typing import List, Union, Any
import tensorflow as tf

class NaiveDense:
    def __init__(self, input_size: int, output_size: int, activation: Any) -> None:
        self.activation = activation
        w_shape = (input_size, output_size)
        weight_matrix = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
        self.W = tf.Variable(weight_matrix)
        b_shape = output_size
        b_init_val = tf.zeros(b_shape)
        self.b = tf.Variable(b_init_val)

    def __call__(self, input: tf.Tensor) -> tf.Tensor:
        return self.activation(tf.matmul(input, self.W) + self.b)

    @property
    def weights(self) -> List[tf.Variable]:
        return [self.W, self.b]

a = NaiveDense(input_size=28 * 28, output_size=512, activation=tf.nn.relu)

class NaiveSequential:
    def __init__(self, layers: List[NaiveDense]) -> None:
        self.layers = layers

    def __call__(self, input: tf.Tensor) -> tf.Tensor:
        x = input
        for layer in self.layers:
            x = layer(x)
        return x
    

# Create the model using NaiveSequential
model = NaiveSequential([
    NaiveDense(input_size=28 * 28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.relu)
])

class BatchGenerator:
    def __init__(self, images: tf.Tensor, labels: tf.Tensor, batch_size: int) -> None:
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size

    def next(self) -> tf.Tensor:
        images_slice = self.images[self.index : self.index + self.batch_size]
        labels_slice = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images_slice, labels_slice

def one_training_step(model: NaiveSequential, images: tf.Tensor, labels: tf.Tensor) -> tf.Tensor:
    with tf.GradientTape() as tape:
        predictions = model(images)
        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(labels, predictions)
        average_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(average_loss, model.weights)
    update_weights(gradients, model.weights)
    return average_loss

def update_weights(gradients: tf.Tensor, weights: tf.Tensor, lr: float = 0.01) -> tf.Tensor:
    for g, w in zip(gradients, model.weights):
        w.assign_sub(lr * g)
    return weights


def fit(
    model: NaiveSequential,
    images: tf.Tensor,
    labels: tf.Tensor,
    epochs: int = 5,
    batch_size: int = 128,) -> None:
    for epoch_counter in range(epochs):
        print(f"Epoch {epoch_counter}")
        batch_generator = BatchGenerator(images, labels, batch_size)
        for batch_counter in range(len(images) // batch_size):
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch)
            if batch_counter % 100 == 0:
                print(f"loss at batch {batch_counter}: {loss:.2f}")




In [34]:
v = tf.Variable(initial_value=tf.random.normal(shape=(3, 1)))
print(v)
v[:,0].assign(1)
print(v)
v = tf.ones((3, 1), dtype=tf.float32)
print(v)

<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[-0.16025944],
       [-1.3368016 ],
       [-0.3758089 ]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[1.],
       [1.],
       [1.]], dtype=float32)>
tf.Tensor(
[[1.]
 [1.]
 [1.]], shape=(3, 1), dtype=float32)


In [39]:
x = tf.constant(3.0)
with tf.GradientTape(persistent=True) as tape:
    tape.watch(x)
    y = x * x
    z = y * y
dz_dx = tape.gradient(z, x)
print(dz_dx)

tf.Tensor(108.0, shape=(), dtype=float32)
